In [1]:
# imports
from itertools import islice
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import pprint
from riotwatcher import LolWatcher, ApiError
my_region = 'na1'
from api_key import API_KEY

lol_watcher = LolWatcher(API_KEY)

In [2]:
# select my riot puuid
my_region = 'na1'
me = lol_watcher.summoner.by_name(my_region, 'slayer56429')
puid = me["puuid"]
print(puid)

XixSHIdrUeBgVQHvWxw8nXK3yY_ybNbj5oeR36b5Db4T93o35j1bJ1sxgO1l1QW4DDHJyDTSnL4mjQ


In [3]:
# find last 100 ranked games - start time currently indicates latest season at the end of 2022
match_hist = lol_watcher.match.matchlist_by_puuid(my_region,puid,count=100, queue = 420,start_time = 1668582000)

In [5]:
# pretty print last game data
last = lol_watcher.match.by_id(my_region,match_hist[0])
# all participant data
participants = last["info"]["participants"]

part_num = 0
for p in participants:
    print(f"---------------------------------------Participant # {part_num}--------------------------------------------")
    part_num+=1
    pprint.pprint(p)
# team data 
teams = last["info"]["teams"]
for t in teams:
    pprint.pprint(t)    

---------------------------------------Participant # 0--------------------------------------------
{'allInPings': 0,
 'assistMePings': 0,
 'assists': 7,
 'baitPings': 0,
 'baronKills': 0,
 'basicPings': 0,
 'bountyLevel': 0,
 'challenges': {'12AssistStreakCount': 0,
                'abilityUses': 213,
                'acesBefore15Minutes': 0,
                'alliedJungleMonsterKills': 0,
                'baronTakedowns': 0,
                'blastConeOppositeOpponentCount': 0,
                'bountyGold': 0,
                'buffsStolen': 0,
                'completeSupportQuestInTime': 0,
                'controlWardsPlaced': 0,
                'damagePerMinute': 1379.883348084213,
                'damageTakenOnTeamPercentage': 0.41352222416286555,
                'dancedWithRiftHerald': 0,
                'deathsByEnemyChamps': 9,
                'dodgeSkillShotsSmallWindow': 5,
                'doubleAces': 0,
                'dragonTakedowns': 1,
                'earliestDragonTak

In [20]:
# define pull current season function which is set to grab the my last 30 games of the newest season which started on Sep 16 2022
current_season_start = []
runes = []

def pull_current_season():

    current_season_start = []
    runes = []

    for m in range(len(match_hist)):
        game = lol_watcher.match.by_id(my_region,match_hist[m])

        for g in game["info"]["participants"]:
            runes.append(g.pop('perks', None))
            if puid in g['puuid']:
                self_game_data = g
                current_season_start.append(self_game_data)

        current_season_df = pd.DataFrame(current_season_start)
        current_season_df = current_season_df.drop(columns=['riotIdName','riotIdTagline','challenges','puuid','summonerId','summonerName'],axis=1)
        current_season_df.to_csv('../data/current_season_first_30.csv',index=False)

In [21]:
pull_current_season()

In [23]:
data = pd.read_csv("../data/current_season_first_30.csv")
pd.set_option('display.max_rows', None)
data

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,4,24,10,0,0,0,0,11459,14,82,...,0,2,9,0,0,25,4,2,4,False
1,0,24,11,0,1,0,7,21950,18,102,...,1,1,4,0,0,22,5,4,6,True
2,1,18,10,1,1,0,0,12298,14,154,...,1,1,8,0,0,35,8,5,8,False
3,2,3,18,0,2,0,0,17737,17,56,...,3,5,3,0,3,44,9,2,19,True
4,1,26,6,0,0,0,0,10683,13,154,...,1,4,0,0,0,30,6,6,6,True
5,0,19,10,0,1,0,0,12861,14,82,...,1,1,2,0,2,31,6,4,4,True
6,0,7,13,1,0,0,0,12556,14,56,...,1,3,7,0,0,17,3,3,3,False
7,0,0,0,0,0,0,0,320,2,82,...,0,0,0,0,0,1,0,0,1,False
8,0,14,7,0,0,0,2,16859,17,82,...,0,2,5,0,0,28,3,4,4,True
9,0,5,6,0,0,0,0,13777,15,9,...,0,1,10,0,0,33,3,2,1,False
